# Present Value of Liabilities and Funding Ratio

The funding ratio is the ratio of the current value of assets to the present value of the liabilities.

In order to compute the present value, we need to discount the amount of the liability based on the relevant interest rate derived from the yield curve. For simplicity, we'll assume that the yield curve is flat, and so the interest rate is the same for all horizons.



The present value of a set of liabilities $L$ where each liability $L_i$ is due at time $t_i$ is give by:

$$ PV(L) = \sum_{i=1}^{k} B(t_i) L_i$$

where $B(t_i)$ is the price of a pure discount bond that pays 1 dollar at time $t_i$.



If we assume the yield curve is flat and the annual rate of interest is $r$ then $B(t)$ is given by

$$B(t) = \frac{1}{(1+r)^t}$$

In other words, $B$ compounded $t$ times by an $r$ return, equates $1\$$ at such time horizon $t$.


In [1]:
import numpy as np
import pandas as pd
import kit as erk

import ipywidgets as widgets
from IPython.display import display

### Discount rate [B]

In [2]:
def discount(t, r):
    """
    Compute the price of a pure discount bond that pays $1 at time t.
    Where t is in years and r is the annual interest rate.
    """
    return (1+r)**(-t)

b = discount(10, 0.03)
b

0.7440939148967249

If you buy that bond today, and hold it for 10 years at an interest rate of 3 percent per year, we will get paid \$1

In [3]:
b*(1.03**10)

1.0

### Present Value

In [4]:
def present_value(l, r):
    """
    Compute the present value of a list of liabilities given by the time as an index.
    """
    dates = l.index
    discounts = discount(dates, r)
    
    return (discounts*l).sum()

Assume that you have 4 liabilities, of 1, 1.5, 2, and 2.5M dollars. Assume the first of these are 3 years away and the subsequent ones are spaced out 6 months apart, i.e. at time 3, 3.5, 4 and 4.5 years from now. Let's compute the present value of the liabilities based on an interest rate of 3% per year.

In an individual investment context, one can think of liabilities as Goals, such as saving for a down payment for a house, college expenses for children, or retirement income. In each of these cases, we have a requirement of a cash flow at some point in the future. Anytime one have a future cash requirement, it as a liability.

In [5]:
liabilities = pd.Series(data=[1, 1.5, 2, 2.5], index=[3, 3.5, 4, 4.5])
present_value(liabilities, 0.03)

6.233320315080045

So, in present day, one will need 6.23 millions at a 3% interest rate to pay liabilities in the 4.5 years horizon.

### Funding Ratio

In [6]:
def funding_ratio(assets, liabilities, r):
    """
    Computes the funding ratio of a series of liabilities, based on an interest rate and current value of assets
    """
    return assets / present_value(liabilities, r)

funding_ratio(5, liabilities, 0.03)

0.8021407126958777

This means that with 5 millions, we can cover 80% of our debt; granted a 3% interest rate.


When interest rates drop down, liabilities tend to get more expensive, or in other words, our present value grows slower in time and is unable to generate enough wealth to cover liabilities in due time. 
Assume interest rates go down to 2%.

In [7]:
funding_ratio(5, liabilities, 0.02)

0.7720304366941648

We can examine the effect of interest rates on funding ratio too.

In [8]:
def show_funding_ratio(assets, r):
    fr = funding_ratio(assets, liabilities, r)

    print(f'Funding Ratio: {fr*100:.2f}%')
    

controls = widgets.interactive(    show_funding_ratio,
                                   assets = widgets.IntSlider(min=1, max=10, step=1, value=5),
                                   r= (0, 0.20, 0.01)
                            )

display(controls)

interactive(children=(IntSlider(value=5, description='assets', max=10, min=1), FloatSlider(value=0.1, descript…

As the illustration above shows, even if your assets do not go down in value, cash can be a risky asset if you think about the funding ratio rather than the asset value. Even though cash is a "safe asset" in the sense that the asset value does not go down, cash can be a very risky asset because the value of the liabilities goes up when interest rates go down. Therefore, if you think about your savings in terms of funding ratio (i.e. how much money do you have compared to what you need) then cash is a risky asset and can result in a decline in your funding ratio.